# Planification d'Emplois du Temps Universitaires

## Introduction

Ce projet vise à appliquer les méthodes de programmation par contraintes (CSP) pour résoudre le problème de la planification des emplois du temps universitaires. Nous utiliserons un solveur CSP pour générer des emplois du temps valides en tenant compte de diverses contraintes.

## Modélisation du Problème

### Variables
- Créneaux horaires
- Salles
- Cours/Examens
- Enseignants

### Contraintes
- Exclusion mutuelle
- Capacité des salles
- Disponibilité des enseignants
- Préférences

## Implémentation avec un Solveur CSP

### Choix du Solveur
- MiniZinc ou OR-Tools

### Installation et Configuration

In [1]:
# Exemple d'installation du solveur
!pip install ortools

  Obtaining dependency information for ortools from https://files.pythonhosted.org/packages/41/df/18e3fbb6082e1c0367595aa4e482c464c189e4bf8a8ca7adf86e402d2c96/ortools-9.12.4544-cp39-cp39-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for absl-py>=2.0.0 from https://files.pythonhosted.org/packages/a2/ad/e0d3c824784ff121c03cc031f944bc7e139a8f1870ffd2845cc2dd76f6c4/absl_py-2.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for numpy>=1.13.3 from https://files.pythonhosted.org/packages/96/ff/06d1aa3eeb1c614eda245c1ba4fb88c483bee6520d361641331872ac4b82/numpy-2.0.2-cp39-cp39-macosx_14_0_arm64.whl.metadata
  Using cached numpy-2.0.2-cp39-cp39-macosx_14_0_arm64.whl.metadata (60 kB)
  Obtaining dependency information for pandas>=2.0.0 from https://files.pythonhosted.org/packages/9c/b9/5cead4f63b6d31bdefeb21a679bc5a7f4aaf262ca7e07e2bc1c341b68470/pandas-2.2.3-cp39-cp39-macosx_11_0_arm64.whl.metadata
  Using cached pandas-2.2.3-cp39-cp39-macosx_11_0_arm64.whl.met

### Définition des Variables et Contraintes


In [2]:
from ortools.sat.python import cp_model

# Création du modèle
model = cp_model.CpModel()

# Définir les variables et les contraintes ici
# Exemple de définition de variables
# Créneaux horaires: 0 à 23 pour représenter les heures de la journée
time_slots = [model.NewIntVar(0, 23, f'time_slot_{i}') for i in range(24)]

# Salles: 0 à 9 pour représenter 10 salles différentes
rooms = [model.NewIntVar(0, 9, f'room_{i}') for i in range(10)]

# Cours/Examens: 0 à 4 pour représenter 5 cours/examens différents
courses = [model.NewIntVar(0, 4, f'course_{i}') for i in range(5)]

# Enseignants: 0 à 4 pour représenter 5 enseignants différents
teachers = [model.NewIntVar(0, 4, f'teacher_{i}') for i in range(5)]

# Exemple de définition de contraintes
# Exclusion mutuelle entre certains créneaux horaires
model.AddNoOverlap([time_slots[2], time_slots[3], time_slots[4]])

# Capacité des salles
model.Add(rooms[0] > 30)

# Disponibilité des enseignants
model.Add(teachers[0] != 3)

# Préférences
model.Add(courses[0] > 2)

TypeError: NotSupported: model.get_interval_index('IntVar')

## Résolution du Problème

### Exécution du Solveur

In [ ]:
# Exemple de résolution du problème
solver = cp_model.CpSolver()
status = solver.Solve(model)


### Analyse des Solutions
- Analyser et visualiser les solutions générées.

## Optimisation

### Critères d'Optimisation
- Minimiser les conflits
- Maximiser la satisfaction des préférences

### Comparaison des Solutions
- Avant et après optimisation

## Extensions et Tests

### Contraintes Supplémentaires
- Intégrer des règles de répartition géographique ou des préférences spécifiques.

### Tests avec Données Réelles
- Utiliser des données réelles pour tester la robustesse du modèle.

## Conclusion

- Résumé des résultats obtenus.
- Perspectives pour des améliorations futures.